In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy.stats as stats
import pprint

np.set_printoptions(precision=3, suppress=True)  # easier to read

In [2]:
# Getting names and values for all categorical columns.
train_df = pd.read_csv('training.csv', index_col='id')
cat_cols = [i for i in train_df if train_df[i].nunique() < 30]
num_cols = [i for i in train_df if i not in cat_cols and i != 'rank' and 'id' not in i]
for col in cat_cols:
    print("    '{}': [{}],".format(col, ', '.join(map(lambda x: "'"+str(x)+"'", train_df[col].unique()))))

    'app_type': ['other', 'forum', 'wiki', 'calendar', 'poll', 'social'],
    'owner_type': ['space', 'user'],
    'is_mentions_me': ['0', '1'],
    'is_mentions_connections': ['0', '1'],
    'is_commented_by_me': ['0', '1'],
    'is_commented_by_connections': ['0', '1'],
    'is_liked_by_me': ['0', '1'],
    'is_liked_by_connections': ['1', '0'],
    'poster_gender': ['male', 'female'],
    'poster_is_employee': ['1', '0'],
    'poster_is_lead': ['0'],
    'poster_is_in_connections': ['1', '0'],
    'poster_focus': ['marketing', 'other', 'engineering', 'sales', 'none'],
    'poster_order': ['6'],
    'participant1_action': ['liked', 'commented', 'viewed'],
    'participant1_gender': ['male', 'female'],
    'participant1_is_employee': ['1', '0'],
    'participant1_is_lead': ['0'],
    'participant1_is_in_connections': ['1', '0'],
    'participant1_focus': ['marketing', 'other', 'engineering', 'sales'],
    'participant1_order': ['5'],
    'participant2_action': ['viewed', 'commented', 

In [3]:
CATEGORIES = {\
    'app_type': ['other', 'forum', 'wiki', 'calendar', 'poll', 'social'],
    'owner_type': ['space', 'user'],
    'is_mentions_me': [0, 1],
    'is_mentions_connections': [0, 1],
    'is_commented_by_me': [0, 1],
    'is_commented_by_connections': [0, 1],
    'is_liked_by_me': [0, 1],
    'is_liked_by_connections': [1, 0],
    'poster_gender': ['male', 'female'],
    'poster_is_employee': [1, 0],
    'poster_is_in_connections': [1, 0],
    'poster_focus': ['marketing', 'other', 'engineering', 'sales', 'none'],
    'participant1_action': ['liked', 'commented', 'viewed'],
    'participant1_gender': ['male', 'female'],
    'participant1_is_employee': [1, 0],
    'participant1_is_in_connections': [1, 0],
    'participant1_focus': ['marketing', 'other', 'engineering', 'sales'],
    'participant2_action': ['viewed', 'commented', 'liked'],
    'participant2_gender': ['male', 'female'],
    'participant2_is_employee': [0, 1],
    'participant2_is_in_connections': [1, 0],
    'participant2_focus': ['other', 'marketing', 'sales', 'engineering'],
    'participant3_action': ['viewed', 'commented', 'liked'],
    'participant3_gender': ['male', 'female'],
    'participant3_is_employee': [0, 1],
    'participant3_is_in_connections': [1, 0],
    'participant3_focus': ['marketing', 'other', 'engineering', 'sales'],
    'participant4_action': ['viewed', 'liked', 'commented'],
    'participant4_gender': ['male', 'female'],
    'participant4_is_employee': [0, 1],
    'participant4_is_in_connections': [1, 0],
    'participant4_focus': ['other', 'marketing', 'sales', 'engineering'],
    'participant5_action': ['viewed', 'liked', 'commented'],
    'participant5_gender': ['male', 'female'],
    'participant5_is_employee': [0, 1],
    'participant5_is_in_connections': [1, 0],
    'participant5_focus': ['other', 'marketing', 'sales', 'engineering']
}

In [4]:
# Dictionaries for chosen double and triple crosses.
double_crosses = {'app_type': ['is_commented_by_connections', 'is_liked_by_connections', 
                             'number_of_comments', 'participant1_is_employee', 'participant1_is_in_connections',
                             'participant1_action', 'participant1_focus', 'participant2_focus', 
                             'participant3_focus', 'participant4_focus', 'participant5_focus'],
                
                'is_commented_by_connections': ['app_type', 'participant2_is_employee', 'participant3_is_employee',
                                                'participant1_action', 'participant2_action', 'participant3_action',
                                                'participant4_action', 'participant5_action', 'participant2_focus',
                                                'participant3_focus', 'participant4_focus'],
                
                'is_liked_by_connections': ['is_commented_by_connections', 'participant1_action',
                                            'participant2_action',  'participant3_action', 'participant4_action',
                                            'participant5_action', 'participant1_focus', 'participant3_focus',
                                            'participant4_focus', 'participant1_is_employee', 
                                            'participant2_is_employee', 'participant3_is_employee',
                                            'participant4_is_employee', 'participant5_is_employee'],
                
                'is_mentions_connections': ['is_commented_by_connections', 'participant4_focus',
                                            'participant1_action', 'participant2_action', 'participant3_action',
                                            'participant4_action', 'participant5_action',
                                            'participant2_is_employee', 'participant3_is_employee',
                                            'participant4_is_employee', 'participant5_is_employee'],
                
                'owner_type': ['app_type', 'number_of_comments', 'number_of_likes', 'owner_influence',
                               'participant3_is_employee', 'participant2_action', 'participant3_action',
                               'participant4_action', 'participant5_action'],
                
                'poster_focus': ['is_commented_by_connections', 'owner_influence', 'poster_influence',
                                 'poster_is_employee', 'poster_is_in_connections', 'participant2_action',
                                 'participant1_focus', 'participant4_focus', 'participant2_is_employee'],
                
                'poster_gender': ['participant2_action', 'participant3_action', 
                                  'participant4_action', 'participant4_focus'],
                
                'poster_is_employee': ['is_commented_by_connections', 'participant4_focus', 
                                       'participant1_is_employee', 'participant2_is_employee', 
                                       'participant3_is_employee', 'poster_is_in_connections'],
                
                'poster_is_in_connections': ['is_commented_by_connections', 'participant2_is_employee',
                                             'poster_focus', 'poster_is_employee'],

                # Participant 1
                'participant1_action': ['app_type', 'number_of_comments', 'number_of_likes', 
                                        'is_commented_by_connections', 'is_liked_by_connections', 
                                        'is_mentions_connections', 'participant1_is_employee', 
                                        'participant2_is_employee', 'participant3_is_employee',
                                        'participant4_is_employee', 'participant5_is_employee',
                                        'participant1_is_in_connections', 'participant2_is_in_connections',
                                        'participant3_is_in_connections', 'participant4_is_in_connections',
                                        'participant5_is_in_connections', 'participant1_focus',
                                        'participant2_focus', 'participant3_focus', 'participant4_focus',
                                        'participant5_focus', 'participant1_gender', 'participant2_gender',
                                        'participant3_gender', 'participant4_gender', 'participant5_gender',
                                        'participant2_action', 'participant3_action', 'participant4_action',
                                        'participant5_action', 'participant1_influence', 'participant2_influence',
                                        'participant3_influence', 'participant4_influence', 'participant5_influence'],
                
                'participant1_focus': ['app_type', 'number_of_comments', 'number_of_likes', 
                                       'is_commented_by_connections', 'is_liked_by_connections', 'owner_influence',
                                       'participant1_action', 'participant2_action', 'participant3_action', 
                                       'participant4_action', 'participant1_influence', 'participant2_influence',
                                       'participant3_influence', 'participant4_influence', 'participant5_influence',
                                       'participant1_is_employee', 'participant2_is_employee', 
                                       'participant3_is_employee', 'participant1_is_in_connections',
                                       'participant3_focus', 'participant4_focus', 'participant5_focus'],
                 
                'participant1_gender': ['number_of_likes', 'owner_influence', 'participant4_focus',
                                        'participant5_focus', 'participant1_action', 'participant2_action',
                                        'participant3_action', 'participant4_action', 'participant3_is_employee',
                                        'poster_gender'],
                
                'participant1_is_employee': ['app_type', 'is_commented_by_connections', 'is_liked_by_connections',
                                             'number_of_comments', 'number_of_likes', 'participant1_action',
                                             'participant2_action', 'participant3_action', 'participant4_action',
                                             'participant1_focus', 'participant3_focus', 'participant4_focus',
                                             'participant5_focus', 'participant1_influence', 'participant2_influence',
                                             'participant3_influence', 'participant4_influence', 'participant5_influence', 
                                             'participant2_is_employee', 'participant3_is_employee'],
                
                'participant1_is_in_connections': ['app_type', 'number_of_comments', 'number_of_likes',
                                                   'participant1_action', 'participant2_action', 'participant3_action',
                                                   'participant1_focus', 'participant4_focus', 'participant1_is_employee',
                                                   'participant2_is_employee', 'participant3_is_employee',
                                                   'participant2_influence', 'participant4_influence', 'participant5_influence'],

                # Participant 2
                'participant2_action': ['app_type', 'is_commented_by_connections', 'is_liked_by_connections',
                                        'is_mentions_connections', 'number_of_comments', 'number_of_likes',
                                        'owner_influence', 'participant1_action', 'participant3_action',
                                        'participant4_action', 'participant5_action', 'participant1_focus',
                                        'participant2_focus', 'participant3_focus', 'participant4_focus',
                                        'participant1_influence', 'participant2_influence', 'participant3_influence',
                                        'participant4_influence', 'participant5_influence',
                                        'participant1_is_employee', 'participant2_is_employee', 
                                        'participant3_is_employee', 'participant4_is_employee', 
                                        'participant5_is_employee', 'participant2_gender', 'participant3_gender', 
                                        'participant4_gender', 'participant5_gender', 
                                        'participant2_is_in_connections', 'participant3_is_in_connections',
                                        'participant4_is_in_connections', 'participant5_is_in_connections'],
                
                'participant2_focus': ['number_of_comments', 'number_of_likes', 'owner_influence',
                                       'participant1_action', 'participant2_action', 'participant1_influence',
                                       'participant2_influence', 'participant3_influence', 'participant4_influence',
                                       'participant5_influence', 'participant2_is_employee', 
                                       'participant3_is_employee', 'participant4_focus', 'participant5_focus'],
                
                'participant2_gender': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes',
                                        'owner_influence', 'participant1_action', 'participant2_action', 'participant3_action',
                                        'participant4_action', 'participant5_action', 'participant1_influence',
                                        'participant2_influence', 'participant3_influence', 'participant4_influence',
                                        'participant5_influence', 'participant2_is_employee', 'participant3_is_employee',
                                        'participant4_is_employee', 'participant4_focus'],
                
                'participant2_is_employee': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments',
                                             'number_of_likes', 'participant1_action', 'participant2_action',
                                             'participant3_action', 'participant4_action', 'participant5_action',
                                             'participant1_focus', 'participant3_focus', 'participant4_focus',
                                             'participant1_influence', 'participant2_influence', 'participant3_influence',
                                             'participant4_influence', 'participant5_influence', 'participant1_is_employee',
                                             'participant3_is_employee', 'participant4_is_employee'],
                
                'participant2_is_in_connections': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes',
                                                   'participant1_action', 'participant2_action', 'participant3_action',
                                                   'participant4_action', 'participant5_action', 'participant1_influence',
                                                   'participant2_influence', 'participant3_influence', 'participant4_influence',
                                                   'participant5_influence', 'participant3_is_employee', 
                                                   'participant4_is_employee', 'participant4_focus'],
                
                # Participant 3
                'participant3_action': ['app_type', 'is_commented_by_connections', 'is_liked_by_connections',
                                        'is_mentions_connections', 'number_of_comments', 'number_of_likes',
                                        'owner_influence', 'participant1_action', 'participant2_action',
                                        'participant4_action', 'participant5_action', 'participant1_focus',
                                        'participant3_focus', 'participant4_focus', 'participant5_focus',
                                        'participant1_influence', 'participant2_influence', 'participant3_influence',
                                        'participant4_influence', 'participant5_influence', 'participant1_is_employee',
                                        'participant2_is_employee', 'participant3_is_employee', 'participant4_is_employee',
                                        'participant5_is_employee', 'participant3_gender', 'participant4_gender',
                                        'participant5_gender', 'participant3_is_in_connections', 'participant4_is_in_connections',
                                        'participant5_is_in_connections'],
                
                'participant3_focus': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments',
                                       'number_of_likes', 'owner_influence', 'participant1_action', 'participant2_action',
                                       'participant3_action', 'participant4_action', 'participant1_focus', 'participant4_focus',
                                       'participant1_influence', 'participant2_influence', 'participant3_influence',
                                       'participant4_influence', 'participant5_influence', 'participant1_is_employee',
                                       'participant2_is_employee', 'participant3_is_employee'],
                
                'participant3_gender': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes', 'owner_influence',
                                        'participant1_action', 'participant2_action', 'participant3_action', 
                                        'participant4_action', 'participant5_action', 'participant1_influence', 
                                        'participant2_influence', 'participant3_influence', 'participant4_influence',
                                        'participant5_influence', 'participant3_is_employee', 'participant4_is_employee', 
                                        'participant5_is_employee', 'participant4_focus'],
                
                'participant3_is_employee': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments',
                                             'number_of_likes', 'owner_influence', 'participant1_action', 'participant2_action',
                                             'participant3_action', 'participant4_action', 'participant5_action',
                                             'participant1_focus', 'participant3_focus', 'participant4_focus',
                                             'participant5_focus', 'participant1_influence', 'participant2_influence',
                                             'participant3_influence', 'participant4_influence', 'participant5_influence',
                                             'participant1_is_employee', 'participant2_is_employee', 'participant4_is_employee',
                                             'participant5_is_employee'],
                
                'participant3_is_in_connections': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes',
                                                   'participant1_action', 'participant2_action', 'participant3_action',
                                                   'participant4_action', 'participant5_action', 'participant1_focus',
                                                   'participant4_focus', 'participant1_influence', 'participant2_influence',
                                                   'participant3_influence', 'participant4_influence', 'participant5_influence',
                                                   'participant2_is_employee', 'participant3_is_employee', 
                                                   'participant4_is_employee', 'participant5_is_employee'],
               
                # Participant 4
                'participant4_action': ['is_commented_by_connections', 'is_liked_by_connections', 'is_mentions_connections',
                                        'number_of_comments', 'number_of_likes', 'owner_influence', 'participant1_action',
                                        'participant2_action', 'participant3_action', 'participant5_action', 'participant1_focus',
                                        'participant3_focus', 'participant4_focus', 'participant5_focus', 'participant1_influence',
                                        'participant2_influence', 'participant3_influence', 'participant4_influence',
                                        'participant5_influence', 'participant1_is_employee', 'participant2_is_employee',
                                        'participant3_is_employee', 'participant4_is_employee', 'participant5_is_employee',
                                        'participant4_gender', 'participant5_gender', 'participant4_is_in_connections',
                                        'participant5_is_in_connections'],
                
                'participant4_focus': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments',
                                       'number_of_likes', 'owner_influence', 'participant1_action', 'participant2_action',
                                       'participant3_action', 'participant4_action', 'participant5_action', 'participant1_focus',
                                       'participant5_focus', 'participant1_influence', 'participant2_influence', 
                                       'participant3_influence', 'participant4_influence', 'participant5_influence',
                                       'participant1_is_employee', 'participant2_is_employee', 'participant3_is_employee',
                                       'participant4_is_employee', 'participant5_is_employee'],
                
                'participant4_gender': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes', 'owner_influence',
                                        'participant1_action', 'participant2_action', 'participant3_action', 'participant4_action',
                                        'participant5_action', 'participant1_influence', 'participant2_influence', 
                                        'participant3_is_employee', 'participant4_is_employee', 'participant5_is_employee',
                                        'participant4_focus'],
                
                'participant4_is_employee': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments',
                                             'number_of_likes', 'participant1_action', 'participant2_action', 'participant3_action',
                                             'participant4_action', 'participant5_action', 'participant1_influence',
                                             'participant2_influence', 'participant3_influence', 'participant4_influence',
                                             'participant5_influence', 'participant1_is_employee', 'participant2_is_employee', 
                                             'participant3_is_employee', 'participant5_is_employee', 'participant3_focus', 
                                             'participant4_focus'],
                
                'participant4_is_in_connections': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes',
                                                   'participant1_action', 'participant2_action', 'participant3_action', 
                                                   'participant4_action', 'participant5_action', 'participant1_influence',
                                                   'participant2_influence', 'participant3_influence', 'participant4_influence',
                                                   'participant5_influence', 'participant2_is_employee', 'participant3_is_employee',
                                                   'participant4_is_employee', 'participant5_is_employee', 'participant4_focus'],
                
                # Participant 5
                'participant5_action': ['is_commented_by_connections', 'is_liked_by_connections', 'is_mentions_connections', 
                                        'number_of_comments', 'number_of_likes', 'owner_influence', 'participant1_action',
                                        'participant2_action', 'participant3_action', 'participant4_action', 'participant1_focus',
                                        'participant3_focus', 'participant4_focus', 'participant5_focus', 'participant1_influence',
                                        'participant2_influence', 'participant3_influence', 'participant4_influence',
                                        'participant5_influence', 'participant1_is_employee', 'participant2_is_employee',
                                        'participant3_is_employee', 'participant4_is_employee', 'participant5_is_employee',
                                        'participant5_gender', 'participant5_is_in_connections'],
                
                'participant5_focus': ['number_of_comments', 'number_of_likes', 'owner_influence', 'participant1_action',
                                       'participant2_action', 'participant3_action', 'participant1_influence',
                                       'participant2_influence', 'participant3_influence', 'participant4_influence',
                                       'participant5_influence', 'participant1_is_employee', 'participant3_is_employee',
                                       'participant2_focus', 'participant4_focus'],
                
                'participant5_gender': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes', 
                                        'participant1_action', 'participant2_action', 'participant3_action', 'participant4_action',
                                        'participant5_action', 'participant1_influence', 'participant2_influence', 
                                        'participant3_influence', 'participant4_influence', 'participant5_influence',
                                        'participant3_is_employee', 'participant4_is_employee', 'participant5_is_employee',
                                        'participant4_focus'],
                
                'participant5_is_employee': ['is_commented_by_connections', 'is_liked_by_connections', 'number_of_comments', 
                                             'number_of_likes', 'participant1_action', 'participant2_action', 'participant3_action',
                                             'participant4_action', 'participant5_action', 'participant1_influence',
                                             'participant2_influence', 'participant3_influence', 'participant4_influence',
                                             'participant5_influence', 'participant2_is_employee', 'participant3_is_employee',
                                             'participant4_is_employee', 'participant3_focus', 'participant4_focus', 
                                             'participant5_focus'],
                
                'participant5_is_in_connections': ['is_commented_by_connections', 'number_of_comments', 'number_of_likes',
                                                   'participant1_action', 'participant2_action', 'participant3_action', 
                                                   'participant4_action', 'participant5_action', 'participant1_influence',
                                                   'participant2_influence', 'participant3_influence', 'participant4_influence',
                                                   'participant5_influence', 'participant3_is_employee', 'participant4_is_employee',
                                                   'participant5_is_employee', 'participant4_focus']}

triple_crosses = \
{'app_type': {'is_liked_by_connections': ['participant1_action',
                                          'participant2_action',
                                          'participant3_action',
                                          'participant4_action',
                                          'participant3_focus'],
              'participant1_is_employee': ['participant1_focus',
                                           'participant2_action',
                                           'participant2_influence'],
              'participant1_is_in_connections': ['number_of_comments'],
              'participant2_action': ['is_commented_by_connections',
                                      'is_liked_by_connections',
                                      'number_of_comments',
                                      'participant1_action',
                                      'participant3_action',
                                      'participant4_action',
                                      'participant5_action',
                                      'participant1_focus',
                                      'participant3_focus',
                                      'participant4_focus',
                                      'participant2_influence',
                                      'participant3_influence',
                                      'participant4_influence',
                                      'participant5_influence'],
              'participant3_action': ['number_of_comments',
                                      'participant1_action',
                                      'participant2_action',
                                      'participant4_action',
                                      'participant5_action',
                                      'participant1_is_employee',
                                      'participant2_is_employee',
                                      'participant3_focus',
                                      'participant4_focus']},
 'is_commented_by_connections': {'participant1_action': ['participant2_action',
                                                         'participant3_action',
                                                         'participant3_is_employee',
                                                         'participant4_is_employee',
                                                         'participant5_focus'],
                                 'participant2_action': ['is_liked_by_connections',
                                                         'number_of_comments',
                                                         'number_of_likes',
                                                         'participant1_focus',
                                                         'participant1_is_employee',
                                                         'participant2_is_employee',
                                                         'participant3_is_employee',
                                                         'participant4_is_employee',
                                                         'participant5_is_employee',
                                                         'participant2_influence',
                                                         'participant3_influence',
                                                         'participant4_influence',
                                                         'participant5_influence',
                                                         'participant3_action',
                                                         'participant4_action',
                                                         'participant5_action'],
                                 'participant3_action': ['number_of_comments',
                                                         'participant1_is_employee',
                                                         'participant2_action',
                                                         'participant4_action',
                                                         'participant5_action',
                                                         'participant3_focus',
                                                         'participant4_focus'],
                                 'participant3_focus': ['is_liked_by_connections',
                                                        'participant2_action',
                                                        'participant4_action',
                                                        'participant2_is_employee',
                                                        'participant3_is_employee'],
                                 'participant4_focus': ['is_liked_by_connections',
                                                        'participant2_action',
                                                        'participant3_action',
                                                        'participant4_action',
                                                        'participant5_action',
                                                        'participant3_is_employee',
                                                        'participant4_is_employee'],
                                 'participant5_action': ['number_of_likes',
                                                         'participant2_action',
                                                         'participant3_action',
                                                         'participant4_action',
                                                         'participant2_is_employee',
                                                         'participant3_focus',
                                                         'participant4_focus',
                                                         'participant5_focus']},
 'is_liked_by_connections': {'participant1_action': ['is_commented_by_connections',
                                                     'number_of_comments',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant3_influence',
                                                     'participant4_influence',
                                                     'participant5_influence'],
                             'participant1_focus': ['is_commented_by_connections',
                                                    'participant1_is_employee',
                                                    'participant2_is_employee',
                                                    'participant3_is_employee',
                                                    'participant2_action',
                                                    'participant3_action',
                                                    'participant3_focus'],
                             'participant1_is_employee': ['participant1_focus',
                                                          'participant3_focus',
                                                          'participant2_action',
                                                          'participant3_action',
                                                          'participant2_is_employee',
                                                          'participant3_is_employee'],
                             'participant2_action': ['is_commented_by_connections',
                                                     'number_of_comments',
                                                     'participant1_action',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee',
                                                     'participant5_is_employee',
                                                     'participant2_influence',
                                                     'participant3_influence',
                                                     'participant4_influence',
                                                     'participant5_influence',
                                                     'participant3_focus',
                                                     'participant4_focus'],
                             'participant2_is_employee': ['participant2_action'],
                             'participant3_action': ['is_commented_by_connections',
                                                     'number_of_comments',
                                                     'participant1_action',
                                                     'participant2_action',
                                                     'participant5_action',
                                                     'participant1_is_employee',
                                                     'participant4_is_employee',
                                                     'participant3_focus',
                                                     'participant4_focus'],
                             'participant3_focus': ['participant1_is_employee',
                                                    'participant2_is_employee',
                                                    'participant2_action',
                                                    'participant4_action'],
                             'participant3_is_employee': ['participant2_is_employee'],
                             'participant4_action': ['is_commented_by_connections',
                                                     'participant1_action',
                                                     'participant2_action',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant4_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant5_is_employee'],
                             'participant4_focus': ['is_commented_by_connections',
                                                    'participant1_is_employee',
                                                    'participant3_is_employee',
                                                    'participant2_action',
                                                    'participant3_action',
                                                    'participant5_action'],
                             'participant5_action': ['is_commented_by_connections',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant2_is_employee',
                                                     'participant3_focus',
                                                     'participant4_focus',
                                                     'participant5_focus']},
 'is_mentions_connections': {'is_commented_by_connections': ['participant1_action',
                                                             'participant2_action',
                                                             'participant3_action'],
                             'participant1_action': ['number_of_comments',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee',
                                                     'participant5_focus'],
                             'participant2_action': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     ' number_of_comments',
                                                     ' number_of_likes',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant2_influence',
                                                     'participant3_influence',
                                                     'participant4_influence',
                                                     'participant5_influence'],
                             'participant3_action': ['is_commented_by_connections',
                                                     'number_of_comments',
                                                     'participant2_action',
                                                     'participant3_focus',
                                                     'participant4_focus'],
                             'participant3_is_employee': ['is_commented_by_connections',
                                                          'number_of_comments',
                                                          'participant2_action',
                                                          'participant4_action'],
                             'participant4_focus': ['participant2_action',
                                                    'participant4_action',
                                                    'participant5_action'],
                             'participant4_is_employee': ['participant2_action',
                                                          'participant5_action']},
 'owner_type': {'participant2_action': ['is_commented_by_connections',
                                        'is_liked_by_connections',
                                        'number_of_comments',
                                        'number_of_likes',
                                        'participant1_action',
                                        'participant4_action',
                                        'participant5_action',
                                        'participant1_focus',
                                        'participant3_focus',
                                        'participant4_focus',
                                        'participant1_is_employee',
                                        'participant3_is_employee',
                                        'participant4_is_employee',
                                        'participant5_is_employee',
                                        'participant2_influence',
                                        'participant3_influence',
                                        'participant4_influence',
                                        'participant5_influence'],
                'participant5_action': ['is_commented_by_connections',
                                        'number_of_likes',
                                        'participant2_action',
                                        'participant3_action',
                                        'participant2_influence',
                                        'participant3_influence',
                                        'participant4_influence',
                                        'participant5_influence',
                                        'participant2_is_employee',
                                        'participant3_is_employee',
                                        'participant4_is_employee',
                                        'participant5_is_employee',
                                        'participant3_focus',
                                        'participant5_focus']},
 'participant1_action': {'is_commented_by_connections': ['participant2_action',
                                                         'participant3_action',
                                                         'participant3_focus'],
                         'is_liked_by_connections': ['participant3_focus',
                                                     'participant4_focus'],
                         'is_mentions_connections': ['is_commented_by_connections',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant3_is_employee'],
                         'participant2_action': ['is_commented_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant4_action'],
                         'participant3_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'participant2_action',
                                                 'participant3_focus'],
                         'participant3_is_employee': ['is_liked_by_connections',
                                                      'number_of_comments',
                                                      'number_of_likes',
                                                      'participant3_focus'],
                         'participant4_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'number_of_likes',
                                                      'participant2_action',
                                                      'participant2_is_employee'],
                         'participant5_focus': ['number_of_comments',
                                                'number_of_likes',
                                                'participant1_is_employee']},
 'participant1_focus': {'is_commented_by_connections': ['participant1_action',
                                                        'participant2_action',
                                                        'participant3_action',
                                                        'participant4_focus'],
                        'is_liked_by_connections': ['participant1_action',
                                                    'participant2_action',
                                                    'participant1_is_employee',
                                                    'participant4_focus'],
                        'participant1_is_employee': ['is_commented_by_connections',
                                                     'participant1_influence',
                                                     'participant2_action',
                                                     'participant3_action'],
                        'participant2_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'number_of_comments',
                                                'number_of_likes',
                                                'participant1_action',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant5_action',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant5_is_employee'],
                        'participant2_is_employee': ['participant2_action',
                                                     'participant4_focus'],
                        'participant3_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'number_of_comments',
                                                'participant1_action',
                                                'participant2_action',
                                                'participant4_action',
                                                'participant5_action',
                                                'participant1_is_employee',
                                                'participant4_focus'],
                        'participant3_focus': ['is_liked_by_connections',
                                               'participant1_is_employee',
                                               'participant2_is_employee',
                                               'participant3_is_employee',
                                               'participant2_action',
                                               'participant4_action'],
                        'participant3_is_employee': ['participant2_action',
                                                     'participant4_focus'],
                        'participant4_focus': ['is_commented_by_connections',
                                               'participant1_is_employee',
                                               'participant2_is_employee',
                                               'participant3_is_employee',
                                               'participant4_is_employee',
                                               'participant3_action',
                                               'participant4_action',
                                               'participant5_action']},
 'participant1_gender': {'participant1_action': ['participant3_action',
                                                 'participant5_focus'],
                         'participant2_action': ['number_of_likes',
                                                 'participant2_influence',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_action',
                                                 'participant4_action',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_action': ['is_commented_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant5_action',
                                                 'participant2_influence',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_is_employee': ['participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant3_focus'],
                         'participant4_action': ['number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant5_action',
                                                 'participant2_influence',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant4_focus': ['participant2_action',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant5_action'],
                         'participant5_focus': ['participant3_is_employee']},
 'participant1_is_employee': {'is_commented_by_connections': ['participant1_action',
                                                              'participant2_action',
                                                              'participant4_focus'],
                              'is_liked_by_connections': ['participant1_action',
                                                          'participant2_action',
                                                          'participant4_action',
                                                          'participant4_focus'],
                              'participant1_focus': ['participant1_influence',
                                                     'participant2_action',
                                                     'participant3_action'],
                              'participant2_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant2_influence',
                                                      'participant3_influence',
                                                      'participant4_influence',
                                                      'participant5_influence',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant3_action',
                                                      'participant4_action'],
                              'participant2_is_employee': ['participant2_action',
                                                           'participant4_focus'],
                              'participant3_action': ['participant1_action',
                                                      'participant2_action',
                                                      'participant4_action',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant3_focus': ['participant3_is_employee'],
                              'participant3_is_employee': ['participant2_action',
                                                           'participant4_focus'],
                              'participant4_focus': ['is_commented_by_connections',
                                                     'participant2_influence',
                                                     'participant3_influence',
                                                     'participant4_influence',
                                                     'participant5_influence',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action']},
 'participant1_is_in_connections': {'app_type': ['is_liked_by_connections',
                                                 ' participant1_is_employee',
                                                 'participant2_action',
                                                 'participant3_action'],
                                    'participant1_focus': ['number_of_comments',
                                                           'number_of_likes',
                                                           'participant1_influence',
                                                           'participant3_influence',
                                                           'participant1_is_employee',
                                                           'participant3_action'],
                                    'participant1_is_employee': ['is_commented_by_connections',
                                                                 'participant1_focus'],
                                    'participant2_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_comments',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant3_action',
                                                            'participant4_action',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee'],
                                    'participant3_action': ['participant1_action',
                                                            'participant2_action',
                                                            'participant3_focus',
                                                            'participant4_focus']},
 'participant2_action': {'is_commented_by_connections': ['participant1_action',
                                                         'participant3_action',
                                                         'participant5_action',
                                                         'participant4_focus'],
                         'is_liked_by_connections': ['participant1_action',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant1_focus',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                         'is_mentions_connections': ['is_commented_by_connections',
                                                     'participant1_action',
                                                     'participant3_action',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                         'participant1_action': ['is_commented_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant3_action',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee'],
                         'participant1_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant3_action'],
                         'participant1_is_employee': ['is_commented_by_connections',
                                                      'participant1_focus',
                                                      'participant4_focus',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant3_action'],
                         'participant2_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant3_action',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                         'participant3_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant1_is_employee',
                                                 'participant4_is_employee',
                                                 'participant4_influence',
                                                 'participant5_influence'],
                         'participant3_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_action',
                                                'participant1_focus',
                                                'participant2_is_employee',
                                                'participant3_is_employee'],
                         'participant3_is_employee': ['is_commented_by_connections',
                                                      'participant2_is_employee',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant4_focus'],
                         'participant4_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant5_action',
                                                 'participant1_focus',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant4_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant4_is_employee',
                                                'participant3_action',
                                                'participant4_action'],
                         'participant4_is_employee': ['is_commented_by_connections',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'number_of_likes',
                                                 'participant3_action',
                                                 'participant4_focus',
                                                 'participant5_focus',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant4_focus',
                                                      'participant5_focus']},
 'participant2_focus': {'participant2_action': ['is_commented_by_connections',
                                                'number_of_comments',
                                                'number_of_likes',
                                                'participant1_action',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant1_focus',
                                                'participant3_focus',
                                                'participant2_influence',
                                                'participant3_influence',
                                                'participant4_influence',
                                                'participant5_influence',
                                                'participant2_is_employee',
                                                'participant3_is_employee'],
                        'participant2_is_employee': ['participant2_action',
                                                     'participant2_influence',
                                                     'participant4_focus'],
                        'participant4_focus': ['is_commented_by_connections',
                                               'participant1_focus',
                                               'participant2_influence',
                                               'participant3_influence',
                                               'participant4_influence',
                                               'participant5_influence',
                                               'participant2_is_employee',
                                               'participant3_is_employee',
                                               'participant4_is_employee',
                                               'participant3_action',
                                               'participant4_action',
                                               'participant5_action'],
                        'participant5_focus': ['participant1_action',
                                               'participant2_influence',
                                               'participant3_influence',
                                               'participant5_influence']},
 'participant2_gender': {'participant2_is_employee': ['participant2_action',
                                                      'participant3_action',
                                                      'participant4_focus'],
                         'participant3_action': ['is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant4_action',
                                                 'participant5_action',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence'],
                         'participant3_is_employee': ['participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action'],
                         'participant4_action': ['is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant3_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence'],
                         'participant4_is_employee': ['participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant4_focus'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant5_focus',
                                                 'participant3_influence',
                                                 'participant4_influence',
                                                 'participant5_influence']},
 'participant2_is_employee': {'is_commented_by_connections': ['participant1_action',
                                                              'participant2_action',
                                                              'participant3_action'],
                              'is_liked_by_connections': ['participant1_action',
                                                          'participant3_action',
                                                          'participant4_action',
                                                          'participant1_focus',
                                                          'participant3_focus',
                                                          'participant3_is_employee'],
                              'participant1_focus': ['is_commented_by_connections',
                                                     'number_of_comments',
                                                     'number_of_likes',
                                                     'participant1_is_employee',
                                                     'participant3_is_employee',
                                                     'participant3_action'],
                              'participant2_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant1_focus',
                                                      'participant4_focus',
                                                      'participant1_is_employee',
                                                      'participant3_is_employee'],
                              'participant3_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant1_action',
                                                      'participant2_action',
                                                      'participant4_action',
                                                      'participant1_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant3_is_employee': ['participant2_action',
                                                           'participant5_action',
                                                           'participant3_focus'],
                              'participant4_focus': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant1_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant4_action'],
                              'participant4_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant2_action',
                                                           'participant4_focus'],
                              'participant5_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant2_action',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant5_focus']},
 'participant2_is_in_connections': {'participant2_action': ['is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant3_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence'],
                                    'participant3_action': ['is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant4_action',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence',
                                                            'participant3_focus',
                                                            'participant4_focus'],
                                    'participant3_is_employee': ['participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant4_focus'],
                                    'participant4_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_comments',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant2_action',
                                                            'participant3_action',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence'],
                                    'participant4_is_employee': ['participant2_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant4_focus'],
                                    'participant5_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_comments',
                                                            'number_of_likes',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant5_focus',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence']},
 'participant3_action': {'is_commented_by_connections': ['participant1_action',
                                                         'participant2_action',
                                                         'participant5_action',
                                                         'participant3_focus',
                                                         'participant4_focus'],
                         'is_liked_by_connections': ['participant1_action',
                                                     'participant2_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                         'participant1_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee'],
                         'participant1_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_focus',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee'],
                         'participant2_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'participant1_action',
                                                 'participant4_action',
                                                 'participant5_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee'],
                         'participant2_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_focus',
                                                      'participant4_focus',
                                                      'participant2_action',
                                                      'participant5_action',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                         'participant3_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_action',
                                                'participant4_action',
                                                'participant1_focus',
                                                'participant2_is_employee',
                                                'participant3_is_employee'],
                         'participant3_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant4_action',
                                                      'participant5_action'],
                         'participant4_action': ['is_commented_by_connections',
                                                 'number_of_likes',
                                                 'participant2_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant4_influence',
                                                 'participant5_influence'],
                         'participant4_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_focus',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee'],
                         'participant4_is_employee': ['is_commented_by_connections',
                                                      'participant4_action',
                                                      'participant5_action'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant4_influence',
                                                 'participant5_influence',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant4_action',
                                                      'participant5_action']},
 'participant3_focus': {'is_commented_by_connections': ['participant1_action',
                                                        'participant2_action',
                                                        'participant3_action',
                                                        'participant4_focus'],
                        'is_liked_by_connections': ['participant1_action',
                                                    'participant2_action',
                                                    'participant3_action',
                                                    'participant4_action',
                                                    'participant5_action',
                                                    'participant4_focus'],
                        'participant1_action': ['is_commented_by_connections',
                                                'number_of_comments',
                                                'participant3_action'],
                        'participant1_focus': ['is_commented_by_connections',
                                               'is_liked_by_connections',
                                               'participant1_is_employee',
                                               'participant2_is_employee',
                                               'participant3_is_employee',
                                               'participant3_action',
                                               'participant4_focus'],
                        'participant1_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant4_focus',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant3_action'],
                        'participant2_action': ['is_commented_by_connections',
                                                'number_of_comments',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant5_is_employee',
                                                'participant2_influence',
                                                'participant3_influence',
                                                'participant4_influence',
                                                'participant5_influence',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant5_action'],
                        'participant2_is_employee': ['participant2_action',
                                                     'participant3_action',
                                                     'participant5_action',
                                                     'participant4_focus'],
                        'participant3_is_employee': ['is_liked_by_connections',
                                                     'participant2_action',
                                                     'participant4_focus'],
                        'participant4_action': ['is_liked_by_connections',
                                                'number_of_comments',
                                                'number_of_likes',
                                                'participant1_action',
                                                'participant2_action',
                                                'participant5_action',
                                                'participant1_focus',
                                                'participant4_focus',
                                                'participant1_is_employee',
                                                'participant4_influence',
                                                'participant5_influence']},
 'participant3_gender': {'participant2_action': ['participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee'],
                         'participant3_action': ['is_liked_by_connections',
                                                 'participant1_action',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_is_employee': ['participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                         'participant4_action': ['is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant4_is_employee': ['is_liked_by_connections',
                                                      'participant2_is_employee',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action'],
                         'participant5_action': ['is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant5_is_employee': ['participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant5_focus']},
 'participant3_is_employee': {'is_commented_by_connections': ['participant1_action',
                                                              'participant2_action',
                                                              'participant3_action',
                                                              'participant5_action',
                                                              'participant3_focus'],
                              'is_liked_by_connections': ['participant1_action',
                                                          'participant3_action',
                                                          'participant4_action',
                                                          'participant5_action',
                                                          'participant1_focus',
                                                          'participant3_focus',
                                                          'participant2_is_employee',
                                                          'participant4_is_employee'],
                              'participant2_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant1_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant2_is_employee': ['is_liked_by_connections',
                                                           'participant1_focus',
                                                           'participant4_focus',
                                                           'participant2_action',
                                                           'participant5_action',
                                                           'participant4_is_employee'],
                              'participant3_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee',
                                                      'participant2_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant3_focus': ['is_commented_by_connections',
                                                     'participant1_action',
                                                     'participant4_action',
                                                     'participant1_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_influence',
                                                     'participant4_influence',
                                                     'participant5_influence'],
                              'participant4_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_action',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant2_is_employee',
                                                      'participant3_focus'],
                              'participant4_focus': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant4_is_employee',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action'],
                              'participant4_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant2_is_employee'],
                              'participant5_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_is_employee',
                                                      'participant3_action',
                                                      'participant3_focus',
                                                      'participant5_focus'],
                              'participant5_is_employee': ['is_commented_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action',
                                                           'participant2_is_employee',
                                                           'participant4_focus',
                                                           'participant5_focus']},
 'participant3_is_in_connections': {'participant2_action': ['is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence'],
                                    'participant3_action': ['is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence',
                                                            'participant3_focus',
                                                            'participant4_focus'],
                                    'participant4_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant4_influence',
                                                            'participant5_influence'],
                                    'participant4_is_employee': ['is_commented_by_connections',
                                                                 'is_liked_by_connections',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee'],
                                    'participant5_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant5_focus'],
                                    'participant5_is_employee': ['is_commented_by_connections',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant3_is_employee',
                                                                 'participant5_focus']},
 'participant4_action': {'is_commented_by_connections': ['participant1_action',
                                                         'participant2_action',
                                                         'participant3_action',
                                                         'participant5_action',
                                                         'participant3_focus',
                                                         'participant4_focus'],
                         'is_liked_by_connections': ['participant1_action',
                                                     'participant2_action',
                                                     'participant5_action',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant4_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                         'participant1_action': ['is_commented_by_connections',
                                                 'is_mentions_connections',
                                                 'number_of_comments',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee'],
                         'participant1_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant3_focus',
                                                'participant4_focus'],
                         'participant1_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_focus',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee'],
                         'participant2_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'participant1_action',
                                                 'participant5_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant2_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_focus',
                                                      'participant4_focus',
                                                      'participant2_action',
                                                      'participant5_action',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                         'participant3_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_action',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee'],
                         'participant3_is_employee': ['is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                         'participant4_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee'],
                         'participant4_is_employee': ['is_commnted_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant4_focus'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_likes',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant5_focus'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant4_focus',
                                                      'participant5_focus']},
 'participant4_focus': {'is_commented_by_connections': ['participant1_action',
                                                        'participant2_action',
                                                        'participant5_action',
                                                        'participant3_focus'],
                        'is_liked_by_connections': ['participant1_action',
                                                    'participant3_action',
                                                    'participant4_action',
                                                    'participant5_action',
                                                    'participant1_focus',
                                                    'participant3_focus'],
                        'participant1_focus': ['is_commented_by_connections',
                                               'is_liked_by_connections',
                                               'participant1_is_employee',
                                               'participant2_is_employee',
                                               'participant3_is_employee',
                                               'participant3_action',
                                               'participant3_focus'],
                        'participant1_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant3_action'],
                        'participant2_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'number_of_comments',
                                                'participant1_action',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant5_action',
                                                'participant1_focus',
                                                'participant3_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant5_is_employee'],
                        'participant2_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant5_action',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                        'participant3_action': ['is_liked_by_connections',
                                                'participant1_action',
                                                'participant2_action',
                                                'participant4_action',
                                                'participant5_action',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant4_is_employee',
                                                'participant5_is_employee',
                                                'participant3_focus'],
                        'participant3_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant2_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant2_is_employee',
                                                     'participant4_is_employee',
                                                     'participant5_is_employee',
                                                     'participant3_focus'],
                        'participant4_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_action',
                                                'participant2_action',
                                                'participant3_action',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_focus'],
                        'participant4_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant2_action',
                                                     'participant2_is_employee'],
                        'participant5_action': ['is_liked_by_connections',
                                                'number_of_comments',
                                                'participant2_action',
                                                'participant4_action',
                                                'participant2_is_employee',
                                                'participant3_focus',
                                                'participant5_focus']},
 'participant4_gender': {'participant3_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant4_action',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_is_employee': ['is_commented_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee'],
                         'participant4_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant3_action',
                                                 'participant5_action',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant4_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant2_action',
                                                'participant3_action',
                                                'participant4_action',
                                                'participant5_action'],
                         'participant4_is_employee': ['participant2_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant4_focus'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant2_action',
                                                 'participant3_action',
                                                 'participant4_action',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant5_focus'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant4_focus',
                                                      'participant5_focus']},
 'participant4_is_employee': {'is_commented_by_connections': ['participant1_action',
                                                              'participant2_action',
                                                              'participant3_action',
                                                              'participant3_focus'],
                              'is_liked_by_connections': ['participant1_action',
                                                          'participant2_action',
                                                          'participant3_action',
                                                          'participant4_action',
                                                          'participant5_action',
                                                          'participant2_is_employee',
                                                          'participant3_is_employee',
                                                          'participant3_focus'],
                              'participant2_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant1_action',
                                                      'participant4_action',
                                                      'participant1_focus',
                                                      'participant3_focus',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee'],
                              'participant2_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant1_focus',
                                                           'participant4_focus',
                                                           'participant2_action',
                                                           'participant3_is_employee'],
                              'participant3_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant1_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus'],
                              'participant3_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action',
                                                           'participant2_is_employee',
                                                           'participant5_is_employee',
                                                           'participant3_focus'],
                              'participant4_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_action',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant4_focus': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant4_action'],
                              'participant5_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'number_of_likes',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant5_focus'],
                              'participant5_is_employee': ['is_commented_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action',
                                                           'participant5_action',
                                                           'participant2_is_employee',
                                                           'participant3_is_employee',
                                                           'participant4_focus',
                                                           'participant5_focus']},
 'participant4_is_in_connections': {'participant3_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant2_action',
                                                            'participant4_action',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence',
                                                            'participant3_focus',
                                                            'participant4_focus'],
                                    'participant3_is_employee': ['is_commented_by_connections',
                                                                 'is_liked_by_connections',
                                                                 'number_of_comments',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant4_is_employee',
                                                                 'participant5_is_employee',
                                                                 'participant3_focus',
                                                                 'participant4_focus'],
                                    'participant4_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_comments',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant2_action',
                                                            'participant3_action',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee'],
                                    'participant4_is_employee': ['is_commented_by_connections',
                                                                 'is_liked_by_connections',
                                                                 'number_of_comments',
                                                                 'number_of_likes',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant3_is_employee',
                                                                 'participant5_is_employee',
                                                                 'participant4_focus'],
                                    'participant5_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_comments',
                                                            'number_of_likes',
                                                            'participant2_action',
                                                            'participant3_action',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant5_focus'],
                                    'participant5_is_employee': ['is_commented_by_connections',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant3_is_employee',
                                                                 'participant4_is_employee',
                                                                 'participant4_focus',
                                                                 'participant5_focus']},
 'participant5_action': {'is_commented_by_connections': ['participant1_action',
                                                         'participant2_action',
                                                         'participant3_action',
                                                         ' participant3_focus',
                                                         ' participant4_focus'],
                         'is_liked_by_connections': ['participant1_action',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant4_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee'],
                         'participant2_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'is_mentions_connections',
                                                 'participant1_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant2_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant1_focus',
                                                      'participant4_focus',
                                                      'participant2_action',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                         'participant3_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_action',
                                                'participant2_action',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee'],
                         'participant3_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                         'participant4_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'number_of_comments',
                                                 'number_of_likes',
                                                 'participant1_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant4_focus': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'participant1_focus',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant2_action'],
                         'participant4_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant4_focus'],
                         'participant5_focus': ['participant1_action',
                                                'participant1_is_employee',
                                                'participant3_is_employee',
                                                'participant2_focus',
                                                'participant4_focus'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant4_focus']},
 'participant5_focus': {'participant1_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'number_of_comments',
                                                'number_of_likes',
                                                'participant2_action',
                                                'participant3_action',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant4_influence',
                                                'participant5_influence'],
                        'participant1_is_employee': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant3_focus',
                                                     'participant4_focus',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant3_action'],
                        'participant2_focus': ['participant2_action',
                                               'participant2_is_employee'],
                        'participant3_action': ['is_commented_by_connections',
                                                'is_liked_by_connections',
                                                'number_of_comments',
                                                'number_of_likes',
                                                'participant1_is_employee',
                                                'participant2_is_employee',
                                                'participant3_is_employee',
                                                'participant4_is_employee',
                                                'participant5_is_employee',
                                                'participant2_action',
                                                'participant4_action',
                                                'participant5_action',
                                                'participant2_influence',
                                                'participant3_influence',
                                                'participant4_influence',
                                                'participant5_influence',
                                                'participant3_focus',
                                                'participant4_focus'],
                        'participant3_is_employee': ['is_liked_by_connections',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant5_action',
                                                     'participant3_focus',
                                                     'participant4_focus'],
                        'participant4_focus': ['is_liked_by_connections',
                                               'participant1_focus',
                                               'participant1_is_employee',
                                               'participant3_is_employee',
                                               'participant4_is_employee',
                                               'participant2_action',
                                               'participant3_action',
                                               'participant4_action',
                                               'participant5_action']},
 'participant5_gender': {'participant3_action': ['is_liked_by_connections',
                                                 'participant1_action',
                                                 'participant4_action',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus'],
                         'participant3_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant4_is_employee',
                                                      'participant5_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                         'participant4_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'participant1_action',
                                                 'participant2_action',
                                                 'participant3_action',
                                                 'participant1_focus',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant1_is_employee',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee'],
                         'participant4_is_employee': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant5_is_employee',
                                                      'participant4_focus'],
                         'participant5_action': ['is_commented_by_connections',
                                                 'is_liked_by_connections',
                                                 'participant2_action',
                                                 'participant3_action',
                                                 'participant2_is_employee',
                                                 'participant3_is_employee',
                                                 'participant4_is_employee',
                                                 'participant5_is_employee',
                                                 'participant3_focus',
                                                 'participant4_focus',
                                                 'participant5_focus'],
                         'participant5_is_employee': ['is_commented_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant4_focus',
                                                      'participant5_focus']},
 'participant5_is_employee': {'is_commented_by_connections': ['participant1_action',
                                                              'participant2_action',
                                                              'participant3_action',
                                                              'participant3_focus',
                                                              'participant4_focus'],
                              'participant2_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'participant1_action',
                                                      'participant4_action',
                                                      'participant1_focus',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                              'participant2_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant1_focus',
                                                           'participant4_focus',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant3_is_employee',
                                                           'participant4_is_employee'],
                              'participant3_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'number_of_likes',
                                                      'participant1_action',
                                                      'participant4_action',
                                                      'participant5_action',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus'],
                              'participant3_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action',
                                                           'participant2_is_employee',
                                                           'participant4_is_employee',
                                                           'participant3_focus',
                                                           'participant4_focus'],
                              'participant4_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'number_of_comments',
                                                      'number_of_likes',
                                                      'participant1_action',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant5_action',
                                                      'participant1_focus',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant1_is_employee',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee'],
                              'participant4_focus': ['is_commented_by_connections',
                                                     'is_liked_by_connections',
                                                     'participant1_focus',
                                                     'participant1_is_employee',
                                                     'participant2_is_employee',
                                                     'participant3_is_employee',
                                                     'participant4_is_employee',
                                                     'participant2_action',
                                                     'participant3_action',
                                                     'participant4_action'],
                              'participant4_is_employee': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action',
                                                           'participant2_is_employee',
                                                           'participant3_is_employee',
                                                           'participant4_focus'],
                              'participant5_action': ['is_commented_by_connections',
                                                      'is_liked_by_connections',
                                                      'participant2_action',
                                                      'participant3_action',
                                                      'participant4_action',
                                                      'participant2_is_employee',
                                                      'participant3_is_employee',
                                                      'participant4_is_employee',
                                                      'participant3_focus',
                                                      'participant4_focus',
                                                      'participant5_focus'],
                              'participant5_focus': ['participant1_action',
                                                     'participant1_is_employee',
                                                     'participant3_is_employee',
                                                     'participant2_focus',
                                                     'participant4_focus']},
 'participant5_is_in_connections': {'participant3_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'number_of_likes',
                                                            'participant1_action',
                                                            'participant2_action',
                                                            'participant4_action',
                                                            'participant5_action',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant2_influence',
                                                            'participant3_influence',
                                                            'participant4_influence',
                                                            'participant5_influence',
                                                            'participant3_focus',
                                                            'participant4_focus'],
                                    'participant3_is_employee': ['is_commented_by_connections',
                                                                 'is_liked_by_connections',
                                                                 'number_of_comments',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant4_is_employee',
                                                                 'participant5_is_employee',
                                                                 'participant3_focus',
                                                                 'participant4_focus'],
                                    'participant4_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'participant1_focus',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant1_is_employee',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee'],
                                    'participant4_focus': ['is_commented_by_connections',
                                                           'is_liked_by_connections',
                                                           'participant1_focus',
                                                           'participant1_is_employee',
                                                           'participant2_is_employee',
                                                           'participant3_is_employee',
                                                           'participant4_is_employee',
                                                           'participant2_action',
                                                           'participant3_action',
                                                           'participant4_action'],
                                    'participant4_is_employee': ['is_commented_by_connections',
                                                                 'is_liked_by_connections',
                                                                 'number_of_comments',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant3_is_employee',
                                                                 'participant5_is_employee',
                                                                 'participant4_focus'],
                                    'participant5_action': ['is_commented_by_connections',
                                                            'is_liked_by_connections',
                                                            'participant2_is_employee',
                                                            'participant3_is_employee',
                                                            'participant4_is_employee',
                                                            'participant5_is_employee',
                                                            'participant3_focus',
                                                            'participant4_focus',
                                                            'participant5_focus'],
                                    'participant5_is_employee': ['is_commented_by_connections',
                                                                 'participant2_action',
                                                                 'participant3_action',
                                                                 'participant4_action',
                                                                 'participant5_action',
                                                                 'participant2_is_employee',
                                                                 'participant3_is_employee',
                                                                 'participant4_is_employee',
                                                                 'participant4_focus',
                                                                 'participant5_focus']},
 'poster_focus': {'participant1_focus': ['is_commented_by_connections',
                                         'is_liked_by_connections',
                                         'number_of_likes',
                                         'participant1_influence',
                                         'participant3_influence',
                                         'participant1_is_employee',
                                         'participant2_is_employee',
                                         'participant3_is_employee',
                                         'participant2_action',
                                         'participant3_action']},
 'poster_gender': {'participant2_action': ['is_commented_by_connections',
                                           'is_liked_by_connections',
                                           'number_of_comments',
                                           'number_of_likes',
                                           'participant1_action',
                                           'participant5_action',
                                           'participant1_focus',
                                           'participant3_focus',
                                           'participant4_focus',
                                           'participant1_is_employee',
                                           'participant2_is_employee',
                                           'participant3_is_employee',
                                           'participant2_influence',
                                           'participant3_influence',
                                           'participant4_influence',
                                           'participant5_influence'],
                   'participant3_action': ['is_commented_by_connections',
                                           'is_liked_by_connections',
                                           'number_of_comments',
                                           'number_of_likes',
                                           'participant1_action',
                                           'participant2_influence',
                                           'participant3_influence',
                                           'participant4_influence',
                                           'participant5_influence',
                                           'participant1_is_employee',
                                           'participant2_is_employee',
                                           'participant3_is_employee',
                                           'participant4_is_employee',
                                           'participant5_is_employee'],
                   'participant4_action': ['is_commented_by_connections',
                                           'is_liked_by_connections',
                                           'number_of_comments',
                                           'number_of_likes',
                                           'participant1_action',
                                           'participant2_influence',
                                           'participant3_influence',
                                           'participant4_influence',
                                           'participant5_influence',
                                           'participant2_is_employee',
                                           'participant3_is_employee',
                                           'participant4_is_employee',
                                           'participant5_is_employee']},
 'poster_is_employee': {'participant1_is_employee': ['participant2_action',
                                                     'participant3_action',
                                                     'participant3_focus'],
                        'participant2_is_employee': ['participant2_action',
                                                     'participant3_action',
                                                     'participant5_action'],
                        'participant3_is_employee': ['participant2_action',
                                                     'participant3_action',
                                                     'participant4_action',
                                                     'participant5_action',
                                                     'participant3_focus']},
 'poster_is_in_connections': {'poster_focus': ['poster_influence',
                                               'poster_is_employee',
                                               'participant3_is_employee']}}

In [5]:
def cartesian_product_of_series(series_1, series_2, df):
    return df.apply(lambda row: (row[series_1], row[series_2]), axis=1)
    
cartesian_product_of_series('app_type', 'is_commented_by_connections', train_df.iloc[:5])

id
175339    (other, 0)
175348    (forum, 1)
175353    (forum, 1)
175371    (other, 0)
175372    (other, 0)
dtype: object

In [6]:
def create_double_feature_crosses(double_crosses_vocab, df):
    res_df = pd.DataFrame()
    used_pairs = []
    for feature_1, val in double_crosses_vocab.items():
        for feature_2 in val:
            if (feature_2, feature_1) not in used_pairs and feature_2 in cat_cols:
                name = feature_1 + '_x_' + feature_2
                res_df[name] = cartesian_product_of_series(feature_1, feature_2, df)
    return res_df

old_train_df = train_df
train_df = create_double_feature_crosses(double_crosses, train_df)

In [7]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
onehot_features = [i for i in cat_cols if 'is_' not in i and 'order' not in i]

encoder.fit(train_df[onehot_features].to_numpy())
rez = encoder.transform(train_df[onehot_features].to_numpy())
rez_df = pd.DataFrame(rez.toarray())
    
rez_df.index = train_df.index

new_feature_names = encoder.get_feature_names(onehot_features)
rename_to_new = {old: new for old, new in zip(list(range(len(new_feature_names))), new_feature_names)}
rez_df = rez_df.rename(rename_to_new, axis=1)

rez_df.head()

KeyError: "None of [Index(['app_type', 'owner_type', 'poster_gender', 'poster_focus',\n       'participant1_action', 'participant1_gender', 'participant1_focus',\n       'participant2_action', 'participant2_gender', 'participant2_focus',\n       'participant3_action', 'participant3_gender', 'participant3_focus',\n       'participant4_action', 'participant4_gender', 'participant4_focus',\n       'participant5_action', 'participant5_gender', 'participant5_focus'],\n      dtype='object')] are in the [columns]"

In [ ]:
train_enc_df = train_df.copy()
train_enc_df = train_enc_df.merge(rez_df, right_index=True, left_index=True, how='outer')
train_enc_df = train_enc_df.drop(onehot_features, axis=1)

train_enc_df.head()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

X = crossed_train_df.drop('rank', axis=1)
y = train_df['rank']

model = ExtraTreesClassifier()
model.fit(X, y)
print(model.feature_importances_)

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(crossed_train_df.drop('rank', axis=1), 
                                                    train_df['rank'], random_state=42)

# ANOVA SVM-C
# 1) anova filter, take 3 best ranked features
anova_filter = SelectKBest(f_regression, k=3)
# 2) svm
clf = svm.LinearSVC()

#anova_svm = make_pipeline(anova_filter, clf)
#anova_svm.fit(X_train, y_train)
#y_pred = anova_svm.predict(X_test)
#print(classification_report(y_test, y_pred))

#coef = anova_svm[:-1].inverse_transform(anova_svm['linearsvc'].coef_)
#print(coef)

anova_filter.fit_transform(X_train, y_train)